# Feistel Cipher

PyPI package - [Feistel Cipher package](https://pypi.org/project/feistelcipher/)

GitHub repo - [Feistel Cipher repo](https://github.com/mayur7garg/FeistelCipher)

Created by - Mayur Kr. Garg

## Importing the necessary libraries

In [1]:
import feistelcipher

print(f"{'Version: ':10}{feistelcipher.__version__}")
print(f"{'Author: ':10}{feistelcipher.__author__}")

Version:  0.2.0
Author:   Mayur Kr. Garg


In [2]:
import feistelcipher.FeistelCipher as fc
import feistelcipher.CryptFunctions as cfs
import feistelcipher.StandardCryptFunctions as scf

import random as rnd
import pandas as pd

## Implementing Feistel Cipher

### Using the inbuilt `StandardCryptFunctions`

#### Numbers to encrypt

In [3]:
numsToEncrypt = [rnd.randint(-99_999, 99_999) for i in range(10)]
numsToEncrypt

[28642, 36237, -31007, -37915, 31467, 50123, 5334, 64223, -69200, 1016]

#### Creating a cryptographic function list

In [4]:
funcList = cfs.CryptFunctions()

#### Adding functions to the `CryptFunctions` object using the inbuilt `StandardCryptFunctions`

The `addFunc` method on a `CryptFunctions` object accepts two arguments:

1. A function which accepts its first argument as an integer
2. A list\tuple of keys corresponding to other arguments of that function

> Kindly note that:
- The second argument is optional and not required when the function has only one parameter.
- The same function can be added multiple times with either same or different keys.
- Kindly note that currently keyword arguments are not supported in the `CryptFunctions` object.

Here, we add all functios from the inbuilt `StandardCryptFunctions` using random keys. 

In [5]:
funcList.addFunc(scf.identity)
funcList.addFunc(scf.add, [rnd.randint(-10, 10)])
funcList.addFunc(scf.multiply, [rnd.randint(-10, 10)])
funcList.addFunc(scf.strLength)
funcList.addFunc(scf.power, [rnd.randint(-10, 10)])
funcList.addFunc(scf.reverse)
funcList.addFunc(scf.truncate, [rnd.randint(-10, 10)])
funcList.addFunc(scf.remainder, [rnd.randint(-10, 10)])
funcList.addFunc(scf.quotient, [rnd.randint(-10, 10)])
funcList.addFunc(scf.scaledDistance, [rnd.randint(-10, 10) for i in range(4)])

#### Creating a `FeistelCipher` object using the `CryptFunctions` object

A Feistel Cipher is uniquely identified by the list of functions (including the order and their associated keys) used to create that cipher. So to be able to decrypt the numbers encoded using this cipher, you may want to save either the `CryptFunctions` object (here the `funcList` variable) or the `FeistelCipher` object (here the `cipher` variable) in a file from wherein it can be retrieved again.

In [6]:
cipher = fc.FeistelCipher(funcList)
cipher.printCipherBlock()


Function                        Keys

identity                        []
add                             [-1]
multiply                        [-2]
strLength                       []
power                           [-3]
reverse                         []
truncate                        [2]
remainder                       [-10]
quotient                        [6]
scaledDistance                  [-9, -10, -7, -6]



#### Encryption

To encrypt a number, just call the `encrypt` method on the `FeiselCipher` object with the number to encrypt as an argument.

Every number is encrypted as an `EncryptedObject` which is denoted by 3 integer values:
- Left
- Right
- BlockLength

> All of these three variables are used during the decryption process

In [7]:
encryptedNums = list(map(cipher.encrypt, numsToEncrypt))
encryptedNums

[EncryptedObject(346, -1766, 3),
 EncryptedObject(379, -475, 3),
 EncryptedObject(34, -27, 3),
 EncryptedObject(3346, -1316, 3),
 EncryptedObject(813, 522, 3),
 EncryptedObject(963, 127, 3),
 EncryptedObject(126, 86, 2),
 EncryptedObject(153, 319, 3),
 EncryptedObject(803, -305, 3),
 EncryptedObject(53, 11, 2)]

#### NOTE

The absolute magnitude of the `left` and the `right` variables of the `EncryptedObject` depends on the list of functions being used in the cipher and is usually hard to predict in advance.

The general rule of thumb is that if the functions used in the cipher generate an output which is usually bigger than the input in terms of absolute magnitude, then the value of the `left` and the `right` variables will explode (such as the `power` function in the inbuilt `StandardCryptFunctions`). 

To combat this, it is a better idea to use some or all functions whose output is generally smaller than the input (such as the `strLength`, `truncate` or `remainder` functions in the inbuilt `StandardCryptFunctions`)

The value of the `blockLength` variable of the `EncryptedObject` depends on the absolute magnitude of the number being encrypted. Since this variable gives some information about the original number, Feistel Cipher should preferably be used only for numbers much bigger in magnitude to prevent brute force attacks.

#### Decryption

To decrypt a number, just call the `decrypt` method on the `FeiselCipher` object with the corresponding `EncryptedObject` variable as an argument.

In [8]:
decryptedNums = list(map(cipher.decrypt, encryptedNums))
decryptedNums

[28642, 36237, -31007, -37915, 31467, 50123, 5334, 64223, -69200, 1016]

#### Visualising the entire data

In [9]:
encryptedLeft = [enc.left for enc in encryptedNums]
encryptedRight = [enc.right for enc in encryptedNums]
encryptedBlockLength = [enc.blockLength for enc in encryptedNums]

df = pd.DataFrame({
    'Numbers': numsToEncrypt,
    'Encrypted_Left': encryptedLeft,
    'Encrypted_Right': encryptedRight,
    'Encrypted_BlockLength': encryptedBlockLength,
    'Decrypted_Numbers': decryptedNums
})

df

,Numbers,Encrypted_Left,Encrypted_Right,Encrypted_BlockLength,Decrypted_Numbers
0,28642,346,-1766,3,28642
1,36237,379,-475,3,36237
2,-31007,34,-27,3,-31007
3,-37915,3346,-1316,3,-37915
4,31467,813,522,3,31467
5,50123,963,127,3,50123
6,5334,126,86,2,5334
7,64223,153,319,3,64223
8,-69200,803,-305,3,-69200
9,1016,53,11,2,1016


In [10]:
(df['Numbers'] == df['Decrypted_Numbers']).all()

True

### Using custom functions

#### Creating custom functions

A valid function which can be used in the function list to build the `CryptFunctions` object for the cipher must have the following properties:
- It accepts at least one argument with the first argument being an integer.
- It outputs an integer value for all possible inputs.
- It should always have the same output for the same set of inputs i.e. it should not be dependent on random number generation or time.

> The exact mathematical implementation of the function is irrelevant.

Here we define two basic functions which we will use to build the `CryptFunctions` object:
- `clamp` function which clamps the input number between the two given numbers.
- `sumOfDigits` which sums up the digits in the absolute value of the input number.

In [11]:
def clamp(num: int, numFirst: int = -100, numSecond: int = 100):
    minNum = min(numFirst, numSecond)
    maxNum = max(numFirst, numSecond)
    return max(min(minNum, num), maxNum)

def sumOfDigits(num: int):
    num = str(abs(num))
    return sum(list(map(int, num)))

#### Creating a cryptographic function list

In [12]:
funcList = cfs.CryptFunctions()

#### Adding functions to the `CryptFunctions` object

Here, we add the `clamp` and the `sumOfDigits` functions to the object using different keys.

> The functions from `StandardCryptFunctions` can also be used alongside custom functions.

In [13]:
funcList.addFunc(clamp, [-1_000, 1000])
funcList.addFunc(sumOfDigits)
funcList.addFunc(clamp, [0])
funcList.addFunc(sumOfDigits)
funcList.addFunc(clamp)

#### Creating a `FeistelCipher` object using the `CryptFunctions` object

In [14]:
cipher = fc.FeistelCipher(funcList)
cipher.printCipherBlock()


Function                        Keys

clamp                           [-1000, 1000]
sumOfDigits                     []
clamp                           [0]
sumOfDigits                     []
clamp                           []



#### Encryption

Kindly note that the encrypted values here are different from the ones we got using the first `FeiselCipher`. This is because, as said before, a `FeistelCipher` is uniquely defined by the set and order of its functions and keys.

In [15]:
encryptedNums = list(map(cipher.encrypt, numsToEncrypt))
encryptedNums

[EncryptedObject(362, 30, 3),
 EncryptedObject(773, 38, 3),
 EncryptedObject(1007, -7, 3),
 EncryptedObject(123, -39, 3),
 EncryptedObject(571, 31, 3),
 EncryptedObject(915, 58, 3),
 EncryptedObject(970, 42, 2),
 EncryptedObject(823, 67, 3),
 EncryptedObject(800, -94, 3),
 EncryptedObject(1016, 13, 2)]

#### Decryption

In [16]:
decryptedNums = list(map(cipher.decrypt, encryptedNums))
decryptedNums

[28642, 36237, -31007, -37915, 31467, 50123, 5334, 64223, -69200, 1016]

#### Visualising the entire data

In [17]:
encryptedLeft = [enc.left for enc in encryptedNums]
encryptedRight = [enc.right for enc in encryptedNums]
encryptedBlockLength = [enc.blockLength for enc in encryptedNums]

df = pd.DataFrame({
    'Numbers': numsToEncrypt,
    'Encrypted_Left': encryptedLeft,
    'Encrypted_Right': encryptedRight,
    'Encrypted_BlockLength': encryptedBlockLength,
    'Decrypted_Numbers': decryptedNums
})

df

,Numbers,Encrypted_Left,Encrypted_Right,Encrypted_BlockLength,Decrypted_Numbers
0,28642,362,30,3,28642
1,36237,773,38,3,36237
2,-31007,1007,-7,3,-31007
3,-37915,123,-39,3,-37915
4,31467,571,31,3,31467
5,50123,915,58,3,50123
6,5334,970,42,2,5334
7,64223,823,67,3,64223
8,-69200,800,-94,3,-69200
9,1016,1016,13,2,1016


In [18]:
(df['Numbers'] == df['Decrypted_Numbers']).all()

True

## Upcoming improvements
- Support for Keyword arguments
- Encrypting/Decrypting iterables of integers
- Support for easily saving the `FeistelCipher` object to a pickled or binary file
- Improved Documentation

## Thank You
If you liked this package or found it useful, consider starring the associated GitHub repository.